In [1]:
import keras.layers as KL
from keras.models import Model
import keras.backend as K
import tensorflow as tf
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
import numpy as np
import os
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.datasets import mnist

In [4]:
def buiding_block(filters,block):
    
    # block1 与 block2 的判断
    if block != 0:
        stride = 1
    else:
        stride = 2
        
    def f(x):
        # 主网络
        y = KL.Conv2D(filters=filters,kernel_size=(1,1),strides=stride)(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        y = KL.Conv2D(filters=filters,kernel_size=(3,3),strides=stride,padding='same')(x)
        y = KL.BatchNormalization(axis=3)(y)
        y = KL.Activation('relu')(y)
        
        y = KL.Conv2D(filters=filters * 4,kernel_size=(1,1))(y)
        y = KL.BatchNormalization(axis=3)(y)
        
        if block == 0:
            shortcut = KL.Conv2D(filters = filters *4,kernel_size=(1,1),strides=stride)(x)
            shortcut = KL.BatchNormalization()(shortcut)
        else:
            shortcut = x
            
        #主网络和shortcut相加
        y=KL.Add()(y,shortcut)
        y = KL.Activation('relu')(y)
        return y
    
    return f
        
            
        
        

In [8]:
def ResNet_Extractor(Xtrain,Ytrain,Xtest,Ytest):
    # head stage Top
    # customize your top input
    input = KL.Input([28,28,1])
    x = KL.Conv2D(filters=64,kernel_size=(3,3),padding='same')(input)
    x = KL.BatchNormalization(axis=3)(x)
    x = KL.Activation('relu')(x)
    
    # 分配你的stage中的block
    
    filters = 64
    block = [2,3,3]
    
    for stage,block_num in enumerate(block):
        print('--stage--',stage,'---')
        for block_id in range(block_num):
            print('---block---',block_id,'---')
            x = buiding_block(filters=filters,block=block_id)(x)
        filters *=2
    # x 这里的x就可以是feature map
    
    # behide stage bottom
    x = KL.AveragePooling2D(pool_size=(2,2))(x)
    x = KL.Flatten()(x)
    x = KL.Dense(units = 10,activation = 'softmax')
    
    model = Model(input=inputnput,outputs = x)
    
    model.compile(loss='categorical_crossentropy',
                 optimizer='adam',
                 metrics = ['accuracy'])
    historys = model.fit(
                Xtrain,Ytrain,
                epochs=6,
                batch_size=200,
                verbose=1,
                validation_data=(Xtest,Ytest)
        )
    model.save('resnetMnist.h5')
    
    return model

In [7]:
block = [2,3,3]

for stage,block_num in enumerate(block):
    print('--stage--',stage,'---')
    for block_id in range(block_num):
        print('---block---',block_id,'---')

--stage-- 0 ---
---block--- 0 ---
---block--- 1 ---
--stage-- 1 ---
---block--- 0 ---
---block--- 1 ---
---block--- 2 ---
--stage-- 2 ---
---block--- 0 ---
---block--- 1 ---
---block--- 2 ---


In [18]:
def main():
    (Xtrain,Ytrain),(Xtest,Ytest) = mnist.load_data()
    print(Xtrain.shape)
    Xtrain = Xtrain.reshape(-1,28,28,1)
    Xtest = Xtest.reshape(-1,28,28,1)
    
    Xtrain = Xtrain/255.0
    Xtest = Xtest/255.0
    
    Ytrain = np_utils.to_categorical(Ytrain,10)
    Ytest = np_utils.to_categorical(Ytest,10)
    
    print(Xtrain.shape)
    #ResNet_Extractor(Xtrain,Ytrain,Xtest,Ytest)
